# Test Old Years

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard libraries
import logging
import os
import pathlib
import sys

# 3rd party libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa
from functools import reduce

# Local libraries
import pudl
import pudl.constants as pc

In [3]:
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [4]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_in = pathlib.Path(pudl_settings['pudl_in'])
ds = pudl.workspace.datastore.Datastore(pudl_in, sandbox=True)
#eia861_raw_dfs = pudl.extract.eia861.Extractor(ds).extract([2019])

In [5]:
datapkg_dir = '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/datapkg/pudl-2008/eia-example/'

## Toggle Settings

In [6]:
eia_inputs = {
    "eia860_years": [2009, 2008, 2007, 2006, 2005, 2004],
    "eia860_tables": pudl.constants.pudl_tables["eia860"],
    "eia861_years": [],
    "eia861_tables": pudl.constants.pudl_tables["eia861"],
    "eia923_years": [2009],
    "eia923_tables": pudl.constants.pudl_tables["eia923"],
}

eia860_tables = eia_inputs["eia860_tables"]
eia860_years = eia_inputs["eia860_years"]
eia861_tables = eia_inputs["eia861_tables"]
eia861_years = eia_inputs["eia861_years"]
eia923_tables = eia_inputs["eia923_tables"]
eia923_years = eia_inputs["eia923_years"]

In [7]:
# generate CSVs for the static EIA tables, return the list of tables
static_tables = pudl.etl._load_static_tables_eia(datapkg_dir)

Loading Static EIA Tables fuel_type_eia923 dataframe into CSV
Loading Static EIA Tables prime_movers_eia923 dataframe into CSV
Loading Static EIA Tables fuel_type_aer_eia923 dataframe into CSV
Loading Static EIA Tables energy_source_eia923 dataframe into CSV
Loading Static EIA Tables transport_modes_eia923 dataframe into CSV


In [8]:
# Extract EIA forms 923, 860
eia860_raw_dfs = pudl.extract.eia860.Extractor(ds).extract(eia860_years)
#eia861_raw_dfs = pudl.extract.eia861.Extractor(ds).extract(eia861_years)
eia923_raw_dfs = pudl.extract.eia923.Extractor(ds).extract(eia923_years)

Extracting eia860 spreadsheet data.
Columns for boiler_generator_assn are off: should be 4 but got 6
Columns for generator are off: should be 65 but got 70
Columns for generator_existing are off: should be 76 but got 77
Columns for generator_proposed are off: should be 62 but got 67
Columns for generator_retired are off: should be 75 but got 76
Columns for ownership are off: should be 15 but got 16
Columns for plant are off: should be 47 but got 49
Columns for utility are off: should be 29 but got 31
Extracting eia923 spreadsheet data.


In [9]:
# Test that the column you added made it into the raw table grab here
# Take a look at what the data type is supposed to be and whether it needs transformation

eia860_raw_dfs.keys()
test = eia860_raw_dfs['ownership']
test.columns.tolist()

['fraction_owned',
 'generator_id',
 'operational_status_code',
 'owner_city',
 'owner_name',
 'owner_state',
 'owner_street_address',
 'owner_utility_id_eia',
 'owner_zip_code',
 'plant_id_eia',
 'plant_name_eia',
 'report_year',
 'state',
 'utility_id_eia',
 'utility_name_eia',
 'year']

In [10]:
# Transform EIA forms 860, 861, 923
eia860_transformed_dfs = pudl.transform.eia860.transform(eia860_raw_dfs, eia860_tables=eia860_tables)
#eia861_transformed_dfs = pudl.transform.eia861.transform(eia861_raw_dfs, eia861_tables=eia861_tables)
eia923_transformed_dfs = pudl.transform.eia923.transform(eia923_raw_dfs, eia923_tables=eia923_tables)

Transforming raw EIA 860 DataFrames for ownership_eia860 concatenated across all years.
Transforming raw EIA 860 DataFrames for generators_eia860 concatenated across all years.
Transforming raw EIA 860 DataFrames for plants_eia860 concatenated across all years.
Transforming raw EIA 860 DataFrames for boiler_generator_assn_eia860 concatenated across all years.
Transforming raw EIA 860 DataFrames for utilities_eia860 concatenated across all years.
Transforming raw EIA 923 DataFrames for generation_fuel_eia923 concatenated across all years.
Transforming raw EIA 923 DataFrames for boiler_fuel_eia923 concatenated across all years.
Transforming raw EIA 923 DataFrames for generation_eia923 concatenated across all years.
Transforming raw EIA 923 DataFrames for coalmine_eia923 concatenated across all years.
Transforming raw EIA 923 DataFrames for fuel_receipts_costs_eia923 concatenated across all years.


In [11]:
# See if your transformation fixed the column
#eia860_transformed_dfs['generators_eia860']

In [12]:
# create an eia transformed dfs dictionary
eia_transformed_dfs = eia860_transformed_dfs.copy()
#eia_transformed_dfs.update(eia861_transformed_dfs.copy())
eia_transformed_dfs.update(eia923_transformed_dfs.copy())

In [13]:
# convert types..
eia_transformed_dfs = pudl.helpers.convert_dfs_dict_dtypes(eia_transformed_dfs, 'eia')

/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.8/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.8/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


In [14]:
# See if your datatype worked
#test = eia_transformed_dfs['generators_eia860']
#test.head()

In [15]:
# NOTE: you can't run this twice in a row -- need to reload eia_transformed_dfs
entities_dfs, eia_transformed_dfs = pudl.transform.eia.transform(
    eia_transformed_dfs,
    eia860_years=eia860_years,
    eia923_years=eia923_years,
)

Harvesting IDs & consistently static attributes for EIA plants
Average consistency of static plants values is 98.99%
Harvesting IDs & consistently static attributes for EIA generators
Average consistency of static generators values is 99.94%
Harvesting IDs & consistently static attributes for EIA utilities
Average consistency of static utilities values is 100.00%
Harvesting IDs & consistently static attributes for EIA boilers
Average consistency of static boilers values is 99.92%
Inferring complete EIA boiler-generator associations.
Multiple EIA unit codes:plant_id_eia=10725, unit_id_pudl=1, unit_id_eia=['F801' 'F802']
Multiple EIA unit codes:plant_id_eia=56309, unit_id_pudl=1, unit_id_eia=['G401' 'G402']


In [16]:
entities_dfs = pudl.helpers.convert_dfs_dict_dtypes(entities_dfs, 'eia')

/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.8/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


In [17]:
transformed_dfs = {"Entities": entities_dfs, "EIA": eia_transformed_dfs}

In [18]:
# Double check that the column is still in there
test = transformed_dfs['EIA']['generators_eia860']
test

,report_date,plant_id_eia,generator_id,capacity_mw,fuel_type_code_pudl,multiple_fuels,ownership_code,owned_by_non_utility,deliver_power_transgrid,summer_capacity_mw,...,switch_oil_gas,turbines_inverters_hydrokinetics,nameplate_power_factor,uprate_derate_during_year,uprate_derate_completed_date,current_planned_operating_date,summer_estimated_capability_mw,winter_estimated_capability_mw,retirement_date,utility_id_eia
0,2009-01-01,57348,UNITB,4.3,gas,False,S,<NA>,False,4.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,56687
1,2009-01-01,57348,UNITA,4.3,gas,False,S,<NA>,False,4.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,56687
2,2009-01-01,57347,GE1,102.4,wind,False,S,<NA>,<NA>,102.4,...,NaN,NaN,NaN,NaN,NaN,2010-11-01,102.4,102.4,NaT,56688
3,2009-01-01,57346,1,98.9,wind,False,S,<NA>,True,98.9,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,56683
4,2009-01-01,57345,2,0.9,gas,False,S,<NA>,<NA>,0.9,...,NaN,NaN,NaN,NaN,NaN,2010-10-01,0.9,0.9,NaT,56685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124729,2004-01-01,3,4,403.7,coal,<NA>,S,False,True,362.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,195
124730,2004-01-01,3,3,272.0,coal,<NA>,S,False,True,251.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,195
124731,2004-01-01,3,2,153.1,coal,<NA>,S,False,True,139.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,195
124732,2004-01-01,3,1,153.1,coal,<NA>,S,False,True,138.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,195


In [19]:
# LOAD step
for data_source, transformed_df in transformed_dfs.items():
    pudl.load.csv.dict_dump(transformed_df,
                            data_source,
                            datapkg_dir=datapkg_dir)
# return (
#     list(eia_transformed_dfs.keys())
#     + list(entities_dfs.keys())
#     + static_tables)

Loading Entities plants_entity_eia dataframe into CSV
Loading Entities generators_entity_eia dataframe into CSV
Loading Entities utilities_entity_eia dataframe into CSV
Loading Entities boilers_entity_eia dataframe into CSV
Loading EIA ownership_eia860 dataframe into CSV
Loading EIA generators_eia860 dataframe into CSV
Loading EIA plants_eia860 dataframe into CSV
Loading EIA boiler_generator_assn_eia860 dataframe into CSV
Loading EIA utilities_eia860 dataframe into CSV
Loading EIA generation_fuel_eia923 dataframe into CSV
Loading EIA boiler_fuel_eia923 dataframe into CSV
Loading EIA generation_eia923 dataframe into CSV
Loading EIA coalmine_eia923 dataframe into CSV
Loading EIA fuel_receipts_costs_eia923 dataframe into CSV


In [20]:
transformed_dfs['EIA'].keys()

dict_keys(['ownership_eia860', 'generators_eia860', 'plants_eia860', 'boiler_generator_assn_eia860', 'utilities_eia860', 'generation_fuel_eia923', 'boiler_fuel_eia923', 'generation_eia923', 'coalmine_eia923', 'fuel_receipts_costs_eia923'])

In [21]:
# Aaaaand make sure it loaded properly
test = transformed_dfs['EIA']['ownership_eia860']
test
#test.columns.tolist()

,fraction_owned,generator_id,owner_city,owner_name,owner_state,owner_street_address,owner_utility_id_eia,owner_zip_code,plant_id_eia,utility_id_eia,report_date
0,0.300,GEN1,<NA>,ACE Operating Partners,<NA>,<NA>,34435,NaN,10002,52,2009-01-01
1,0.700,GEN1,<NA>,ACE Power Partners,<NA>,<NA>,34436,NaN,10002,52,2009-01-01
2,0.600,1,<NA>,Alabama Power Co,AL,<NA>,195,NaN,10,195,2009-01-01
3,0.400,1,<NA>,Mississippi Power Co,MS,<NA>,12686,NaN,10,195,2009-01-01
4,0.600,2,<NA>,Alabama Power Co,AL,<NA>,195,NaN,10,195,2009-01-01
...,...,...,...,...,...,...,...,...,...,...,...
21616,0.234,1,<NA>,The Northwestern Mutual Life Ins Co,<NA>,<NA>,50143,NaN,56293,50123,2004-01-01
21617,0.143,1,<NA>,Union Bank of California NA,<NA>,<NA>,50142,NaN,56293,50123,2004-01-01
21618,0.143,1,<NA>,KEF Equity Investment Corp,<NA>,<NA>,49885,NaN,56293,50123,2004-01-01
21619,1.000,GEN1,<NA>,Fortistar LLC,<NA>,<NA>,6571,NaN,54131,50136,2004-01-01


In [130]:
# run: 
# datapkg_to_sqlite PUDL_DIR/datapkg/pudl-2008/eia-example/datapackage.json -c
# Then run the code below

In [90]:
# little helper function
def get_full_sql_table(table_name, engine):
    """Get a full table from a SQL database."""
    # generate table metadata
    md = sa.MetaData()
    md.reflect(engine)
    pt = md.tables
    # make a sql select statement
    select = sa.sql.select([pt[table_name], ])
    # read the sql select into a dataframe
    df = pd.read_sql(select, engine)
    return df
# Generate a SQLalchemy databse engine:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
table_name='utilities_eia860'
table_df = get_full_sql_table(table_name=table_name, engine=pudl_engine)

In [91]:
table_df

,id,utility_id_eia,report_date,street_address,city,state,zip_code,plants_reported_owner,plants_reported_operator,plants_reported_asset_manager,...,entity_type,attention_line,address_2,zip_code_4,contact_firstname,contact_lastname,contact_title,contact_firstname_2,contact_lastname_2,contact_title_2
0,1,62789,2018-01-01,300 Spectrum Center Drive Suit,Irvine,CA,92618,True,None,None,...,Independent Power Producer,None,None,None,None,None,None,None,None,None
1,2,62788,2018-01-01,300 Spectrum Center Drive Suit,Irvine,CA,92618,True,None,None,...,Independent Power Producer,None,None,None,None,None,None,None,None,None
2,3,62787,2018-01-01,"1900 East Golf Road, Suite 103",Schaumburg,IL,60173,True,None,True,...,Independent Power Producer,None,None,None,None,None,None,None,None,None
3,4,62784,2018-01-01,"11550 Ash St, Ste 300",Leawood,KS,66211,True,None,None,...,Independent Power Producer,None,None,None,None,None,None,None,None,None
4,5,62778,2018-01-01,100 California St Suite 400,San Francisco,CA,94118,True,None,None,...,Independent Power Producer,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61182,61183,23,2007-01-01,5527 Paseo Arnaldo,Yorba Linda,CA,92886,None,None,None,...,None,None,None,None,A,Giusiana,Vice President,A,Giusiana,Vice President
61183,61184,21,2007-01-01,None,Panama,OK,74951,None,None,None,...,None,None,P O Box 1740,None,Lundy,Kiger,Vice President,Ruben,Soroeta,President
61184,61185,20,2007-01-01,1001 Nineteenth St 20th Floor,Arlington,VA,22209,None,None,None,...,None,None,None,None,Paul,Burdick,Vice President,None,None,None
61185,61186,8,2007-01-01,2245 Sagamore Parkway North,Lafayette,IN,47904,None,None,None,...,None,None,None,None,Scott A,Market,Utility Resource,William D,Coats,Utilities Manager
